## Q1. Running Elastic

In [1]:
!curl localhost:9200

curl: (7) Failed to connect to localhost port 9200: Connection refused


### Getting the data

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

## Q2. Indexing the data

In [6]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

In [7]:
es_client.info()

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7e404f6a5180>: Failed to establish a new connection: [Errno 111] Connection refused))

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
from tqdm.auto import tqdm

In [ ]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

## Q3. Searching

In [ ]:
query = 'How do I execute a command in a running docker container?'

In [ ]:
search_query = {
    "size": 1,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

In [ ]:
response = es_client.search(index=index_name, body=search_query)

## Q4. Filtering

In [ ]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

get_n_question = es_client.search(index=index_name, body=search_query)

In [ ]:
get_n_question("How do I execute a command in a running docker container?", 3)

## Q5. Building a prompt

In [ ]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [ ]:
context = '\n\n'.join([
    context_template.format(
        question=r['_source']['question'],
        text=r['_source']['text'],
    ) 
    for r in response['hits']['hits']
])

In [ ]:
def rag1(query): 
    search_results = elastic_search_ml(query, 3)
    prompt = build_prompt(query, search_results)
    return prompt

In [ ]:
prompt = rag1("How do I execute a command in a running docker container?")
len(prompt)

## Q6. Tokens

In [ ]:
import tiktoken

In [ ]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [ ]:
len(encoding.encode(prompt))